In [1]:
# Ce script permet de scraper les exemples de formations des métiers de l'Onisep
# Création d'un dictionnaire "metier_formation.json" :
# Métiers : formations

# Création d'un dictionnaire "metier_acces.json" :
# Métiers : accès au métier

import os
import json
import numpy as np
import math
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup

In [2]:
# Chemins utiles
dir_path = '/home/timothee/PycharmProjects/ImpalaPoleEmploi2/Impala/Formations/'
# Dossier data
data_dir_path = os.path.join(dir_path, '0 - data')
# Fichier liste des métiers Onisep
liste_metier_path = os.path.join(data_dir_path, 'Onisep_metier_link.csv')

In [3]:
# Chargement des données
liste_metier = pd.read_csv(liste_metier_path, sep=';', header=0)
liste_metier.head()

,libellé métier,lien site onisep.fr
0,technicien/ne de fabrication de mobilier et de...,http://www.onisep.fr/http/redirection/metier/i...
1,agenceur/euse de cuisines et salles de bains,http://www.onisep.fr/http/redirection/metier/i...
2,chef/fe de fabrication des industries graphiques,http://www.onisep.fr/http/redirection/metier/i...
3,conducteur/trice de machines à imprimer,http://www.onisep.fr/http/redirection/metier/i...
4,opérateur/trice prépresse,http://www.onisep.fr/http/redirection/metier/i...


In [4]:
# Fonction pour récupérer les exemples de formations des métiers de l'Onisep
def get_exemples_formations(link):
    try:
        page = request.urlopen(link).read().decode('utf-8')
        page = page.replace('€', 'euros')  # because encode/decode error
        page = page.replace(u"\u2019", "'")  # because encode/decode error
        page = page.replace(u"\u2026", "é")  # because encode/decode error
        page = page.replace(u"\u0153", "oe")  # because encode/decode error
        page = page.replace('\n', '')  # remove new line
        page = page.replace('  ', '')  # remove big spaces (to be more human readable)

        soup = BeautifulSoup(page, 'html.parser')

        metier = []
        # Onglet "Exemples de formations"
        onglet = soup.find(name='div', attrs={'id': 'oni_onglet-5'})
        if onglet is not None:
            for element in onglet.find_all():
                if element.name=='li':
                    metier.append(element.get_text())
        return(metier)         
    except:
        print('error')
        return('error')
#metier = get_exemples_formations('http://www.onisep.fr/Ressources/Univers-Metier/Metiers/accessoiriste')
#metier

# Fonction pour récupérer les onglets "accès au métier" de l'Onisep
# Fonction pour récupérer les exemples de formations des métiers de l'Onisep
def get_acces_metier(link):
    try:
        page = request.urlopen(link).read().decode('utf-8')
        page = page.replace('€', 'euros')  # because encode/decode error
        page = page.replace(u"\u2019", "'")  # because encode/decode error
        page = page.replace(u"\u2026", "é")  # because encode/decode error
        page = page.replace(u"\u0153", "oe")  # because encode/decode error
        page = page.replace('\n', '')  # remove new line
        page = page.replace('  ', '')  # remove big spaces (to be more human readable)

        soup = BeautifulSoup(page, 'html.parser')

        metier = []
        # Onglet "Exemples de formations"
        onglet = soup.find(name='div', attrs={'id': 'oni_onglet-4'})
        if onglet is not None:
            for element in onglet.find_all(name='p'):
                if element.get_text() not in ['Après la 3e', 'Niveau bac','Niveau bac + 2','Niveau bac + 3',
                                              'Niveau bac + 5','Niveau bac + 8']:
                    metier.append(element.get_text())
        return(metier)         
    except:
        print('error')
        return('error')

metier = get_acces_metier('http://www.onisep.fr/Ressources/Univers-Metier/Metiers/auxiliaire-specialise-specialisee-veterinaire')
metier

["Il existe 2 niveaux de qualification.\xa0 Le titre d'AVQ (auxiliaire vétérinaire qualifié)\xa0se prépare en 1 an et s'adresse aux personnes ayant suivi une scolarité jusqu'en fin de 1re (avec passage en terminale accordé) ou ayant suivi une\xa02de et possédant une expérience professionnelle d'un an minimum, quel que soit le secteur d'activité. Le titre d'ASV (auxiliaire spécialisé vétérinaire) se prépare en 2 ans dont 18 à 23 semaines en centre de formation. Le programme du titre d'ASV comprend des enseignements complémentaires à ceux de l'AVQ : le conseil et la vente, l'assistance à la gestion, aux soins, en chirurgie. 2 options au choix parmi 4 : comptabilité ; animaux de compagnie ; équidés ; animaux de production. Pour accéder à cette formation, il faut\xa0être titulaire d'un baccalauréat ou d'un diplôme de même niveau ou avoir suivi une 2de générale et technologique avec passage accordé en 1re et posséder une expérience professionnnelle de 2 ans, quel que soit le secteur d'activ

In [ ]:
# Dictionnaire des métiers avec les exemples de formations des métiers de l'Onisep
metier_formation = {}
errors = []
for row in liste_metier.itertuples():
    metier = row[1]
    link = row[2]
    exemples_formations = get_exemples_formations(link)
    if exemples_formations != 'error':
        metier_formation[metier] = exemples_formations
    else:
        errors.append(metier)

In [ ]:
len(metier_formation)

In [ ]:
# Copie du dictionnaire au format JSON
output_path = os.path.join(dir_path, 'Scraping Exemples Formations/Exemples_metier_formation.json')
with open(output_path, 'w') as f:
    json.dump(metier_formation, f, indent=4)

In [6]:
# Dictionnaire des métiers avec les onglets "Accès métiers" des métiers de l'Onisep
metier_acces = {}
errors = []
for row in liste_metier.itertuples():
    metier = row[1]
    link = row[2]
    acces_metier = get_acces_metier(link)
    if acces_metier != 'error':
        metier_acces[metier] = acces_metier
    else:
        errors.append(metier)

In [7]:
print(len(metier_acces))
# Copie du dictionnaire au format JSON
output_path = os.path.join(dir_path, 'Scraping Exemples Formations/metier_acces.json')
with open(output_path, 'w') as f:
    json.dump(metier_acces, f, indent=4)

731


In [9]:
metier_acces['accessoiriste']

["Des formations en accessoires et décors du spectacle, du CAP à bac + 2, permettent d'exercer le métier d'accessoiriste. Il est recommandé de compléter ces formations par des expériences sur le terrain : stages professionnels, cours de théâtre en activité extrascolaire, etc. L'objectif est de se constituer un carnet d'adresses, essentiel pour intégrer le milieu. Une base artisanale en menuiserie, ébénisterie, tapisserie, sculpture, soudure, etc. est utile pour être plus polyvalent. Quelques écoles spécialisées (Ensatt, Esad, Ensad...) proposent des diplômes spécifiques au décor qui peuvent constituer un atout pour progresser dans le métier."]